In [1]:
from Solvers import adv_solver, span_solver, span_solver2, span_dual_relax
from Adversary import Adversary, Problem, to_str, visualize
import numpy as np
import matplotlib.pyplot as plt
import itertools
from Examples import exact_k, threshold_k
from ElementDistinctness import ED
from copy import deepcopy as copy
import scipy
import cvxpy as cp
import itertools
import matplotlib as mpl
mpl.rcParams['figure.dpi'] =200
import networkx as nx
import pydot
from networkx.drawing.nx_pydot import graphviz_layout
# import graphviz
# import pygraphviz as pgv # pygraphviz should be available

In [93]:
def powerset(iterable):
    "powerset([1,2,3]) --> () (1,) (2,) (3,) (1,2) (1,3) (2,3) (1,2,3)"
    s = list(iterable)
    return list(itertools.chain.from_iterable(itertools.combinations(s, r) for r in range(len(s)+1)))

def sublist(L, indices):
    sublist = []
    for i in indices:
        sublist.append(L[i])
    return tuple(sublist)
    
def get_flow2_pairs(subset):
    return [(subset[:j] + subset[j+1:], subset[j]) for j in range(len(subset))]

def get_1_certs(problem):
    index_sets = powerset(range(problem.n))
    yes_sets = {indices: set() for indices in index_sets}
    no_sets = {indices: set() for indices in index_sets}
    
    for yes_instance in problem.yes_instances:
        for indices in index_sets:
            yes_sets[indices].add(sublist(yes_instance, indices))
    
    for no_instance in problem.no_instances:
        for indices in index_sets:
            no_sets[indices].add(sublist(no_instance, indices))
    
    for indices in index_sets:
        yes_sets[indices] = yes_sets[indices] - no_sets[indices]

    print('yes', yes_sets)
    print('no',no_sets)
    return yes_sets
    
def remove_tuple(L, v):
    M = list(L)
    M.remove(v)
    return tuple(M)
    

def learning_graph_solver(problem):
    n = problem.n
    K = cp.Variable(nonneg=True)
    index_sets = powerset(range(n))
    print("powerset", index_sets)
    weight_mapping = set()
    for i in range(n+1):
        for indices in itertools.combinations(range(n), i):
            for substring in itertools.combinations_with_replacement(problem.alphabet, i):
                for substring1 in itertools.permutations(substring):
                    weight_mapping.add((indices, substring1))
    print(weight_mapping)
    weight_mapping = {mapping: i for (mapping, i) in zip(weight_mapping, list(range(len(weight_mapping))))}
    print('here')
    print(weight_mapping)
    edges = set()
    for S in index_sets:
        for j in range(n):
            if j not in S:
                edges.add((S, j))
                
    edges = list(edges)
    r = {edge: cp.Variable(problem.yes_len, nonneg=True) for edge in edges}
    p = {edge: cp.Variable(problem.yes_len, nonneg=True) for edge in edges}
    w = {edge: cp.Variable(len(weight_mapping), nonneg=True) for edge in edges}
    constraints = []
    for edge in edges:
        for yes_index in range(problem.yes_len):
            yes = problem.yes_instances[yes_index]
            print(edge)
            constraints.append(
                cp.bmat([
                    [r[edge][yes_index], p[edge][yes_index]],
                     [p[edge][yes_index], w[edge][weight_mapping[(edge[0], sublist(yes, edge[0]))]]]
                    ]) >> 0
            )
    constraints += [
        cp.sum(cp.vstack([r[edge][yes_index] for edge in edges])) <= 1 for yes_index in range(len(problem.yes_instances)) 
    ]
    
    constraints += [
        K >= cp.sum(cp.vstack([w[edge][weight_mapping[(edge[0],sublist(no, edge[0]))]] for edge in edges])) for no in problem.no_instances
    ]
    one_certs = get_1_certs(problem)
    for subset in index_sets:
        for yes_index in range(problem.yes_len):
            yes = problem.yes_instances
            if sublist(yes, subset) not in one_certs[subset] and len(subset) > 0: 
                flow_out = [0]+[p[(subset, j)][yes_index] for j in range(n) if j not in subset]
                flow_in = [0]+[p[(remove_tuple(subset, j), j)][yes_index] for j in subset]
                constraints += [
                    cp.sum(cp.vstack(flow_in)) == cp.sum(cp.vstack(flow_out)) 
                ]
    
    
    constraints += [
        cp.sum(cp.vstack(
            [p[((), j)][yes_index] for j in range(n)
            ])) == 1 for yes_index in range(problem.yes_len)
    ] 
    
    opt_problem = cp.Problem(cp.Minimize(K), constraints)
    opt_problem.solve(verbose=True)
    return {index: p[index].value for index in p}, {index: w[index].value for index in w}              
    
def get_all_sublists(string, sub_len):
    substrings = set()
    for indices in itertools.combinations(range(len(string))):
        substring.append(tuple(string[i] for i in indices))
    return substrings

def dual_learning_graph(problem):
    n = problem.n
    pset = powerset(range(n))
    alpha = {(subset, yes): cp.Variable(nonneg=True) for subset, yes in itertools.product(pset, problem.yes_instances)}
    one_certs = get_1_certs(problem)
    k = {no: cp.Variable(nonneg=True) for no in problem.no_instances}
    substrings = set()

    
    
    

In [94]:
def multilayered_graph(layers, edges):
    # extents = nx.utils.pairwise(itertools.accumulate((0,) + subset_sizes))
    G = nx.DiGraph()
    for i, layer in enumerate(layers):
        G.add_nodes_from(layer, layer=i)
    G.add_edges_from(edges)
    return G

def learning_graph_frame(problem):
    n = problem.n
    one_certs = get_1_certs(problem)
    levels = [
            [to_str(x) for x in itertools.combinations(list(range(n)), i)]
        for i in range(n+1)
    ]
    print(levels)


    edges = []
    for i in range(len(levels)-1):
        curr_level = levels[i]
        next_level = levels[i+1]
        for curr in curr_level:
            for nextl in next_level:
                if len(set(nextl) - set(curr)) == 1:
                    edges.append((curr,nextl))
    G = multilayered_graph(levels, edges)
    pos = nx.multipartite_layout(G, subset_key="layer")
    
    nx.draw(G, pos=pos, with_labels=True)
prob = threshold_k(3, 2)

# learning_graph_frame(prob)



In [95]:
p, w = learning_graph_solver(prob)
# print(get_1_certs(prob))

powerset [(), (0,), (1,), (2,), (0, 1), (0, 2), (1, 2), (0, 1, 2)]
{((1,), (1,)), ((2,), (0,)), ((1, 2), (1, 0)), ((0,), (1,)), ((), ()), ((0, 1, 2), (0, 1, 0)), ((1, 2), (1, 1)), ((0, 1, 2), (1, 1, 0)), ((1, 2), (0, 0)), ((0, 1, 2), (0, 1, 1)), ((0, 2), (1, 0)), ((2,), (1,)), ((0, 1), (1, 0)), ((1,), (0,)), ((1, 2), (0, 1)), ((0, 2), (1, 1)), ((0, 1, 2), (1, 1, 1)), ((0, 1, 2), (0, 0, 0)), ((0, 1), (1, 1)), ((0, 1, 2), (0, 0, 1)), ((0, 2), (0, 0)), ((0, 1), (0, 0)), ((0, 1, 2), (1, 0, 0)), ((0,), (0,)), ((0, 2), (0, 1)), ((0, 1), (0, 1)), ((0, 1, 2), (1, 0, 1))}
here
{((1,), (1,)): 0, ((2,), (0,)): 1, ((1, 2), (1, 0)): 2, ((0,), (1,)): 3, ((), ()): 4, ((0, 1, 2), (0, 1, 0)): 5, ((1, 2), (1, 1)): 6, ((0, 1, 2), (1, 1, 0)): 7, ((1, 2), (0, 0)): 8, ((0, 1, 2), (0, 1, 1)): 9, ((0, 2), (1, 0)): 10, ((2,), (1,)): 11, ((0, 1), (1, 0)): 12, ((1,), (0,)): 13, ((1, 2), (0, 1)): 14, ((0, 2), (1, 1)): 15, ((0, 1, 2), (1, 1, 1)): 16, ((0, 1, 2), (0, 0, 0)): 17, ((0, 1), (1, 1)): 18, ((0, 1, 2), (0

In [81]:
print(p)
print(w)

{((1,), 2): None, ((0, 2), 1): None, ((2,), 1): None, ((), 0): None, ((1, 2), 0): None, ((0,), 1): None, ((1,), 0): None, ((), 2): None, ((2,), 0): None, ((0, 1), 2): None, ((), 1): None, ((0,), 2): None}
{((1,), 2): None, ((0, 2), 1): None, ((2,), 1): None, ((), 0): None, ((1, 2), 0): None, ((0,), 1): None, ((1,), 0): None, ((), 2): None, ((2,), 0): None, ((0, 1), 2): None, ((), 1): None, ((0,), 2): None}


In [21]:
print(())

()


In [24]:
(1,2,3).remove(2)

AttributeError: 'tuple' object has no attribute 'remove'

In [35]:
L = [1,2,3]
L.remove(2)
print(L)

[1, 3]
